# Functions from `ImagePipeline.ipynb`

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

import math

def grayscale(img):
    """NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')
    Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    """
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = draw_lines(img, lines, [255, 0, 0], 2);
    
    return line_img, lines

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    Draw an array of line segments on a blank image
    """
    line_img=np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8);
    if lines is None:
        return line_img
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    return line_img

def slope_thresholding(img,lines):
    """
    This function isolates somewhat vertical line segments.
    Then, it divides raw line segments into left and right lane.
    
    Input: 
    img= Color image from dashcam
    lines= Set of line segments which denote parts of lane lines
    """
    lane_raw=[];
    
    #Check if no raw lines were detected
    if lines is None:
        raw_lane_img= draw_lines(img, lines, [0, 255, 0], 2);
        return raw_lane_img, [], []
    
    #Ignore segments which are somewhat horizantal
    for pts in lines:
        for x1,y1,x2,y2 in pts:
            m=(y2-y1)/(x2-x1)
            if m>.25 or m<-.25:
                lane_raw.append(pts)
    
    #Separate left and right lanes
    left_lane_raw=[];
    right_lane_raw=[];
    for pts in lane_raw:
        for x1,y1,x2,y2 in pts:
            slope=((y2-y1)/(x2-x1));
            if slope<0:
                left_lane_raw.append(pts)
            else:
                right_lane_raw.append(pts)
                
    left_raw_img = draw_lines(img, left_lane_raw, [0, 255, 0], 2)
    right_raw_img = draw_lines(img, right_lane_raw, [0, 0, 255], 2)
    
    
    raw_lane_img = weighted_img(left_raw_img,right_raw_img,1,1,0)
    return raw_lane_img, left_lane_raw, right_lane_raw
    
# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


def draw_lanes(img, left_lane_raw, right_lane_raw):
    """
    This function plots left and right lane lines on dashcam image

    The raw lane lines are aggregated into single line segments denoting lane lines
    """
    
    #Find Top and Bottom x values of the lane
    #y_top=lines[0][0][1]
    #y_bottom=lines[0][0][1]
    #for pts in lines:
    #    for x1,y1,x2,y2 in pts:
    #        if y_top>y1:
    #            y_top=y1
    #        if y_bottom<y1:
    #            y_bottom=y1
    #        if y_top>y2:
    #            y_top=y2
    #        if y_bottom<y2:
    #            y_bottom=y2
    y_bottom=img.shape[0]
    y_top=.6*img.shape[0]

    #Calculate Left lane
    left_x,left_y,left_m,left_c=avg_line(left_lane_raw)
    left_lane=np.array([(y_top-left_c)/left_m,y_top,(y_bottom-left_c)/left_m,y_bottom]);
    left_lane=left_lane.astype(int)

    #Calculate Right lane
    right_x,right_y,right_m,right_c=avg_line(right_lane_raw)
    right_lane=np.array([(y_top-right_c)/right_m,y_top,(y_bottom-right_c)/right_m,y_bottom]);
    right_lane=right_lane.astype(int)
    
    lane=[[left_lane] , [right_lane]]
    if left_lane[0]>10000 or left_lane[0]<0 or right_lane[0]>10000 or right_lane[0]<0:
        return draw_lines(img, []); 
    
    lane_img=draw_lines(img, lane);    
    return lane_img

def avg_line(lines):
    avg_pt=np.array([0,0])
    slope_arr=np.array([]);
    for pts in lines:
        for x1,y1,x2,y2 in pts:
            m=(y2-y1)/(x2-x1)
            slope_arr=np.append(slope_arr,m)
            avg_pt=avg_pt+[(x1+x2)/2.0,(y1+y2)/2.0]
    slope=np.median(slope_arr)
    avg_pt=avg_pt/len(slope_arr)
    x=avg_pt[0]
    y=avg_pt[1]
    intercept= y-slope*x
    
    #print (x,y,slope,intercept)
    return x,y,slope,intercept


def detailImg(Img1, Img2, Img3, Img4, Img5, Img6, Img7, Img8, Img9):

    #Combile all images and show results
    ht=Img1.shape[0]
    wd=Img1.shape[1]
    DetailImg=np.zeros((3*ht, 3*wd, 3), dtype=np.uint8);
    DetailImg[:ht,:wd]=Img1;
    DetailImg[:ht,wd:2*wd]=cv2.merge((Img2,Img2,Img2));
    DetailImg[:ht,2*wd:3*wd]=cv2.merge((Img3,Img3,Img3));
    DetailImg[ht:2*ht,:wd]=cv2.merge((Img4,Img4,Img4));
    DetailImg[ht:2*ht,wd:2*wd]=cv2.merge((Img5,Img5,Img5));
    DetailImg[ht:2*ht,2*wd:3*wd]=Img6;
    DetailImg[2*ht:3*ht,:wd]=Img7;
    DetailImg[2*ht:3*ht,wd:2*wd]=Img8;
    DetailImg[2*ht:3*ht,2*wd:3*wd]=Img9;

    #Add titles for each step
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 2
    color = (255,255,255)
    thickness = 4
    linetype = cv2.LINE_AA
    w_off=10
    h_off=80
    cv2.putText(DetailImg, "1.Input", (w_off,h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "2.Grayscaling", (wd+w_off,h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "3.Blurring", (2*wd+w_off,h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "4.Edge Detection", (w_off,ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "5.Region Masking", (wd+w_off,ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "6.Hough Transform", (2*wd+w_off,ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "7.Slope Thresholding", (w_off,2*ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "8.Line averaging", (wd+w_off,2*ht+h_off),font,scale,color,thickness,linetype)
    cv2.putText(DetailImg, "9.Final Visualization", (2*wd+w_off,2*ht+h_off),font,scale,color,thickness,linetype)


    #Create grid seperators
    cv2.line(DetailImg, (wd, 0), (wd, 3*ht), [255,255,255], 2)
    cv2.line(DetailImg, (2*wd, 0), (2*wd, 3*ht), [255,255,255], 2)
    cv2.line(DetailImg, (0, ht), (3*wd, ht), [255,255,255], 2)
    cv2.line(DetailImg, (0, 2*ht), (3*wd, 2*ht), [255,255,255], 2)
    
    return DetailImg

def pipeline(Img, hyper):
    #STEP 1- INPUT
    Img1=Img

    #STEP 2- Grayscaling
    Img2=grayscale(Img1)

    #STEP 3- Blurring
    Img3=gaussian_blur(Img2, hyper['blur_kernel_size'])

    #STEP 4- Edge detection
    Img4=canny(Img3, hyper['canny_low'], hyper['canny_high'])

    #STEP 5- Region Masking
    Low_Lf=[.05*Img1.shape[1],Img1.shape[0]]
    Up_Lf=[.45*Img1.shape[1],.6*Img1.shape[0]]
    Up_Rt=[.55*Img1.shape[1],.6*Img1.shape[0]]
    Low_Rt=[.95*Img1.shape[1],Img1.shape[0]]
    poly=np.array([[Low_Lf,Up_Lf,Up_Rt,Low_Rt]], dtype=np.int32)
    Img5=region_of_interest(Img4, poly)

    #STEP 6- Hough Transform
    #Detecting line segments from edge pixels and creating blank image with lane lines
    (Img6,lines)=hough_lines(Img5, hyper['hough_rho'], hyper['hough_theta'], hyper['hough_threshold'], 
                             hyper['hough_min_line_len'], hyper['hough_max_line_gap'])

    #STEP 7- Slope Thresholding
    (Img7,left_raw,right_raw)=slope_thresholding(Img6,lines)

    #STEP 8- Averaging Raw lines
    Img8=draw_lanes(Img7,left_raw,right_raw)
    
    #STEP 9- Final Visualization
    #Superimposing lanes on original image
    Img9=weighted_img(Img8, Img1)
    
    #Generate Detail Image
    Img10=detailImg(Img1, Img2, Img3, Img4, Img5, Img6, Img7, Img8, Img9)
    return Img9, Img10
    

Hyperparam={'blur_kernel_size':5,
            'canny_low':50,
            'canny_high':150,
            'hough_rho':2,
            'hough_theta':np.pi/90,
            'hough_threshold':40,
            'hough_min_line_len':30,
            'hough_max_line_gap':20,};

# Real-time Testing

To test my pipeline in realtime, I have printed out an Image of road and pinned it on my study table. The setup is shown in the image below

In [ ]:
# REFERENCE: https://towardsdatascience.com/video-streaming-in-the-jupyter-notebook-635bc5809e85

from ipywebrtc import CameraStream, ImageRecorder
back_camera = CameraStream.facing_environment(audio=False)
image_recorder = ImageRecorder(stream=back_camera)
image_recorder

In [ ]:
from ipywidgets import Widget
Widget.close_all()
test_env=cv2.cvtColor(cv2.imread('test_images/record.png', cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)

fig1 = plt.figure(figsize = (16,16))
ax1 = fig1.add_subplot(111)
ax1.imshow(test_env)
plt.show()

# Displaying Real-time Camera feed in `Jupyter notebook`

Many ideas were tried to display the live feed. However, all except use of `IPython.display` seem to suffer from poor fps performance.

In [ ]:
# REFERENCE: https://github.com/NicksonYap/Jupyter-Webcam/blob/master/Realtime_video_ipython_py3.ipynb

# Import the required modules
import cv2
import numpy as np
from IPython.display import clear_output, display, Image
import PIL.Image
from io import BytesIO


#Use 'jpeg' instead of 'png' (~5 times faster)
def showarray(a, fmt='jpeg'):
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

#%pylab inline 
def plot2(frame): #TOO SLOW 
    imshow(frame)
    show()
    
def plot3(frame): #TOO SLOW 
    plt.imshow(frame)
    display(plt.gcf())

#cv2.startWindowThread()
#cv2.namedWindow("Capture")
def plot4(frame): #No windows
    cv2.imshow("Capture", frame)
    cv2.waitKey(0)

In [ ]:
cam = cv2.VideoCapture(0)
#cam.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
#cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)

for frame_number in range(500):
    #Capture frame
    _, frame = cam.read()
    
    #Perform operation
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame= process_frame(frame)
    
    #Display frame
    showarray(frame)
    # Display the frame until new frame is available
    clear_output(wait=True)

cam.release()

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Benchmarking the real-time Lane-Detection pipeline

In [ ]:
def benchmark_livestream(n_frames):
    cam = cv2.VideoCapture(0)
    #cam.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
    #cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)

    for frame_number in range(n_frames):
        #Capture frame
        _, frame = cam.read()

        #Perform operation
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame= process_frame(frame)

        #Display frame
        showarray(frame)
        # Display the frame until new frame is available
        clear_output(wait=True)
    cam.release()

In [ ]:
import time
start_time = time.time()
n_frames=100;
benchmark_livestream(n_frames)
end_time=time.time();
t= end_time - start_time
print("--- %s seconds ---" % (t))
print("--- %s Average fps ---"% (n_frames/t))

In [ ]:
%timeit benchmark_livestream(n_frames)

In [ ]:
print("--- %s Average fps ---"% (100/7.46))

# Calibrating Camera on Car

Before we can begin with testing our algorithm on a real vehicle, the camera needs to be setup on the dashboard. During this calibration phase, we find a region mask which works well with our algorithm.

In [ ]:
def calibrate_mask(n_frames):
    cam = cv2.VideoCapture(0)

    for frame_number in range(n_frames):
        _, frame = cam.read()

        #Perform operation
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        Low_Lf=[.05*frame.shape[1],frame.shape[0]]
        Up_Lf=[.45*frame.shape[1],.6*frame.shape[0]]
        Up_Rt=[.55*frame.shape[1],.6*frame.shape[0]]
        Low_Rt=[.95*frame.shape[1],frame.shape[0]]
        poly=np.array([[Low_Lf,Up_Lf,Up_Rt,Low_Rt]], dtype=np.int32)
        masked=region_of_interest(frame, poly)
        frame=weighted_img(masked, frame, .2)

        #Display frame
        showarray(frame)
        clear_output(wait=True)
    cam.release()
    
calibrate_mask(100)

# Real-World Test

Finally, I use my surface pro 5 on my 2010 Honda CRV dashboard and test the algorithm in real world conditions

In [ ]:
def process_livestream(n_frames):
    cam = cv2.VideoCapture(0)
    frame_array = []
    Detail_array=[]
    
    for frame_number in range(n_frames):
        #Capture frame
        _, frame = cam.read()

        #Perform operation
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame, Detail= pipeline(frame ,Hyperparam)

        #Display frame
        showarray(frame)
        #Display the frame until new frame is available
        clear_output(wait=True)
        #Save Frame to create a video
        frame_array.append(frame)
        Detail_array.append(Detail)
        
    cam.release()
    
    #Saving Lane-detection Visualization Video
    height , width , layers =  frame.shape
    fps=20
    pathOut='CRV_RealTime_video.mp4'
    video = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, (width,height))
    for i in range(len(frame_array)):
        video.write(cv2.cvtColor(frame_array[i], cv2.COLOR_RGB2BGR))
    video.release()

    #Saving Pipeline Visualization Video
    height , width , layers =  Detail.shape
    pathOut='CRV_RealTime_video_Detail.mp4'
    video = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, (width,height))
    for i in range(len(Detail_array)):
        video.write(cv2.cvtColor(Detail_array[i], cv2.COLOR_RGB2BGR))
    video.release()
    

In [ ]:
process_livestream(600) #FOR 30s CLIPS